# Import Library

In [669]:
# Prevent font from breaking
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'

In [670]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import re
import seaborn as sns
import matplotlib

# Read csv  

In [1133]:
INPUT_PATH = './data/external_open/bigdatamart/7/edit/'
rename = '_edit'

# 사고 다발 지역 정보 / 빅데이터마트 7.안전
oldman = pd.read_csv(f'{INPUT_PATH}oldman{rename}.csv') # 보행노인사고
bicycle = pd.read_csv(f'{INPUT_PATH}bicycle{rename}.csv') # 자전거사고
child = pd.read_csv(f'{INPUT_PATH}child{rename}.csv') # 보행어린이사고
drunk = pd.read_csv(f'{INPUT_PATH}drunk{rename}.csv') # 음주사고 
freezing = pd.read_csv(f'{INPUT_PATH}freezing{rename}.csv') # 결빙사고
jaywalking = pd.read_csv(f'{INPUT_PATH}jaywalking{rename}.csv') # 보행자무단횡단
motorcycle = pd.read_csv(f'{INPUT_PATH}motorcycle{rename}.csv') # 이륜차사고
pedstrians = pd.read_csv(f'{INPUT_PATH}pedstrians{rename}.csv') # 보행자사고 
schoolzone = pd.read_csv(f'{INPUT_PATH}schoolzone{rename}.csv') # 스쿨존내어린이사고
tmzon = pd.read_csv(f'{INPUT_PATH}tmzon{rename}.csv') # 연휴기간 
violt = pd.read_csv(f'{INPUT_PATH}violt{rename}.csv') # 법규위반 
# RiskArea = pd.read_csv(f'{INPUT_PATH}RiskArea{rename}.csv') # 링크기반 
truck = pd.read_csv(f'{INPUT_PATH}truck{rename}.csv') # 화물차 

In [1147]:
# (0) Set required variables
variable_name = 'violt'
prefix = '법규위반'

In [1148]:
# (1) data info check
globals()[f'{variable_name}'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   사고다발지FID  195 non-null    int64  
 1   사고다발지ID   195 non-null    int64  
 2   법규위반      195 non-null    object 
 3   법정동코드     195 non-null    int64  
 4   지점코드      195 non-null    int64  
 5   시도시군구명    195 non-null    object 
 6   지점명       195 non-null    object 
 7   사고건수      195 non-null    int64  
 8   사상자수      195 non-null    int64  
 9   사망자수      195 non-null    int64  
 10  중상자수      195 non-null    int64  
 11  경상자수      195 non-null    int64  
 12  부상신고자수    195 non-null    int64  
 13  경도        195 non-null    float64
 14  위도        195 non-null    float64
 15  다발지역폴리곤   195 non-null    object 
dtypes: float64(2), int64(10), object(4)
memory usage: 24.5+ KB


In [1149]:
# (2) index to list 
# ['사고다발지FID', '사고다발지ID', '법정동코드', '지점코드', '시도시군구명', '지점명', '사고건수', '사상자수', '사망자수', '중상자수', 
# '경상자수', '부상신고자수', '경도', '위도', '다발지역폴리곤']
globals()[f'{variable_name}list'] = globals()[f'{variable_name}'].columns.to_list() 
globals()[f'{variable_name}list']

['사고다발지FID',
 '사고다발지ID',
 '법규위반',
 '법정동코드',
 '지점코드',
 '시도시군구명',
 '지점명',
 '사고건수',
 '사상자수',
 '사망자수',
 '중상자수',
 '경상자수',
 '부상신고자수',
 '경도',
 '위도',
 '다발지역폴리곤']

In [1150]:
# (3) Unusable index drop 
globals()[f'{variable_name}'] = globals()[f'{variable_name}'].drop(columns=['부상신고자수','경도','위도','다발지역폴리곤','시도시군구명','사상자수',
                                                                            # '사고다발지fid', '사고다발지id',
                                                                            '사고다발지FID', '사고다발지ID',
                                                                            '지점코드','법정동코드']).copy()

In [1151]:
globals()[f'{variable_name}']

,법규위반,지점명,사고건수,사망자수,중상자수,경상자수
0,중앙선침범,대구광역시 중구 봉산동(봉산육거리 부근),7,0,3,5
1,중앙선침범,대구광역시 중구 사일동(중앙네거리 부근),4,0,3,3
2,중앙선침범,대구광역시 중구 동인동2가(동인네거리 부근),4,0,1,1
3,중앙선침범,대구광역시 동구 입석동(아양교앞 부근),3,0,0,4
4,중앙선침범,대구광역시 서구 비산동(팔달교 부근),3,0,1,4
...,...,...,...,...,...,...
190,신호위반,대구광역시 수성구 황금동(황금네거리 부근),8,0,2,9
191,신호위반,대구광역시 수성구 두산동(두산오거리 부근),6,0,3,9
192,신호위반,대구광역시 수성구 범어동(범어네거리 부근),6,0,3,2
193,신호위반,대구광역시 달서구 신당동(신당네거리 부근),8,1,8,3


# Edit index

## Only proceed with violt.csv file

In [1152]:
# (4)-0 violt.csv’s check
globals()[f'{variable_name}']['법규위반'].unique()

array(['중앙선침범', '신호위반'], dtype=object)

In [1153]:
# (4)-0-1 Find rows with ‘중앙선침범’ and ‘신호위반’ values ​​and convert to one-hot encoding
globals()[f'{variable_name}']['중앙선침범'] = globals()[f'{variable_name}']['법규위반'].apply(lambda x: 1 if x == '중앙선침범' else 0).fillna(0)
globals()[f'{variable_name}']['신호위반'] = globals()[f'{variable_name}']['법규위반'].apply(lambda x: 1 if x == '신호위반' else 0).fillna(0)

In [1154]:
# (4)-0-2 Delete the '법규위반' column 
globals()[f'{variable_name}'] = globals()[f'{variable_name}'].drop('법규위반', axis=1)

In [1155]:
globals()[f'{variable_name}']

,지점명,사고건수,사망자수,중상자수,경상자수,중앙선침범,신호위반
0,대구광역시 중구 봉산동(봉산육거리 부근),7,0,3,5,1,0
1,대구광역시 중구 사일동(중앙네거리 부근),4,0,3,3,1,0
2,대구광역시 중구 동인동2가(동인네거리 부근),4,0,1,1,1,0
3,대구광역시 동구 입석동(아양교앞 부근),3,0,0,4,1,0
4,대구광역시 서구 비산동(팔달교 부근),3,0,1,4,1,0
...,...,...,...,...,...,...,...
190,대구광역시 수성구 황금동(황금네거리 부근),8,0,2,9,0,1
191,대구광역시 수성구 두산동(두산오거리 부근),6,0,3,9,0,1
192,대구광역시 수성구 범어동(범어네거리 부근),6,0,3,2,0,1
193,대구광역시 달서구 신당동(신당네거리 부근),8,1,8,3,0,1


## Files other than violt.csv file

In [1156]:
# (4) Calculate and add eclo index
globals()[f'{variable_name}'][f'{variable_name}_ECLO'] = globals()[f'{variable_name}'].apply(lambda row: row['사망자수'] * 10 
                                                                                             + row['중상자수'] * 5 + row['경상자수'] * 3, axis=1)

In [1157]:
globals()[f'{variable_name}']

,지점명,사고건수,사망자수,중상자수,경상자수,중앙선침범,신호위반,violt_ECLO
0,대구광역시 중구 봉산동(봉산육거리 부근),7,0,3,5,1,0,30
1,대구광역시 중구 사일동(중앙네거리 부근),4,0,3,3,1,0,24
2,대구광역시 중구 동인동2가(동인네거리 부근),4,0,1,1,1,0,8
3,대구광역시 동구 입석동(아양교앞 부근),3,0,0,4,1,0,12
4,대구광역시 서구 비산동(팔달교 부근),3,0,1,4,1,0,17
...,...,...,...,...,...,...,...,...
190,대구광역시 수성구 황금동(황금네거리 부근),8,0,2,9,0,1,37
191,대구광역시 수성구 두산동(두산오거리 부근),6,0,3,9,0,1,42
192,대구광역시 수성구 범어동(범어네거리 부근),6,0,3,2,0,1,21
193,대구광역시 달서구 신당동(신당네거리 부근),8,1,8,3,0,1,59


In [1158]:
# (5) Rename indexes that may overlap
globals()[f'{variable_name}'] = globals()[f'{variable_name}'].rename(columns={
                                                                              '사고건수': f'{prefix}사고건수',
                                                                              # jaywalking.csv
                                                                              # '발생건수': f'{prefix}발생건수',
                                                                              '사망자수': f'{prefix}사망자수',
                                                                              '중상자수': f'{prefix}중상자수',
                                                                              '경상자수': f'{prefix}경상자수'})

In [1159]:
# (6) index split
globals()[f'{variable_name}'][['시', '구', '동가']] = globals()[f'{variable_name}']['지점명'].str.split(' ',n=2,expand = True)
# (6)-1 Unusable index drop
globals()[f'{variable_name}'] = globals()[f'{variable_name}'].drop("시", axis=1)
# (6)-2 Delete unnecessary elements in the index
globals()[f'{variable_name}']['동가'] = globals()[f'{variable_name}']['동가'].str.split("(").str[0]
globals()[f'{variable_name}']

,지점명,법규위반사고건수,법규위반사망자수,법규위반중상자수,법규위반경상자수,중앙선침범,신호위반,violt_ECLO,구,동가
0,대구광역시 중구 봉산동(봉산육거리 부근),7,0,3,5,1,0,30,중구,봉산동
1,대구광역시 중구 사일동(중앙네거리 부근),4,0,3,3,1,0,24,중구,사일동
2,대구광역시 중구 동인동2가(동인네거리 부근),4,0,1,1,1,0,8,중구,동인동2가
3,대구광역시 동구 입석동(아양교앞 부근),3,0,0,4,1,0,12,동구,입석동
4,대구광역시 서구 비산동(팔달교 부근),3,0,1,4,1,0,17,서구,비산동
...,...,...,...,...,...,...,...,...,...,...
190,대구광역시 수성구 황금동(황금네거리 부근),8,0,2,9,0,1,37,수성구,황금동
191,대구광역시 수성구 두산동(두산오거리 부근),6,0,3,9,0,1,42,수성구,두산동
192,대구광역시 수성구 범어동(범어네거리 부근),6,0,3,2,0,1,21,수성구,범어동
193,대구광역시 달서구 신당동(신당네거리 부근),8,1,8,3,0,1,59,달서구,신당동


In [1160]:
# (6)-3 Check the value of the column and then supplement it
print(globals()[f'{variable_name}']['동가'].unique())

['봉산동' '사일동' '동인동2가' '입석동' '비산동' '대명동' '침산동' '칠성동2가' '태전동' '산격동' '고성동3가'
 '만촌동' '중동' '두산동' '월성동' '죽전동' '대봉동' '신암동' '내당동' '평리동' '범어동' '범물동' '수성동1가'
 '지산동' '수성동4가' '본동' '파호동' '이곡동' '용산동' '포정동' '삼덕동1가' '신천동' '이현동' '봉덕동'
 '감삼동' '상인동' '태평로3가' '남산동' '신서동' '율하동' '황금동' '성당동' '지저동' '용계동' '노원동2가'
 '칠성동1가' '두류동' '송현동' '덕산동' '삼덕동3가' '효목동' '동천동' '수성동2가' '상동' '대곡동' '신당동'
 '호산동' '다사읍' '삼덕동2가' '진천동' '공평동' '중대동' '읍내동' '대현동' '대천동' '장기동' '본리동']


### Only child & oldman files are processed

#### child.csv

In [965]:
# (6)-4-1 child.csv / If there is a dong with the meaning of “부근,” change its value to dong.
globals()[f'{variable_name}']['동가'] = globals()[f'{variable_name}']['동가'].replace({ '상인동 대구월서초등학교 부근': '상인동'})
                                                                                   # ,'달성동 달성공원 부근': '달성동'})

#### oldman.csv

In [930]:
# (6)-4-1 oldman.csv / If there is a dong with the meaning of “부근,” change its value to dong.
globals()[f'{variable_name}']['동가'] = globals()[f'{variable_name}']['동가'].replace({ '효목동 한국전력공사동대구지점 부근' : '효목동'
                                                                                   ,'달성동 달성공원 부근': '달성동'})

### Files other than child & oldman

#### Files other than violt.csv

In [1161]:
# (6)-4-2 Unusable index drop
globals()[f'{variable_name}'] = globals()[f'{variable_name}'].drop("지점명", axis=1)

In [1145]:
# (6)-5-1 Average and initialize the values ​​of variables for rows where ‘동가’ and ‘법정동코드’ are the same
grouped = globals()[f'{variable_name}'].groupby(['구', '동가']).agg({
                                                                    f'{prefix}사고건수': 'sum',
                                                                    # jaywalk.csv
                                                                    # f'{prefix}발생건수': 'sum', 
                                                                    f'{prefix}사망자수': 'sum', 
                                                                    f'{prefix}중상자수': 'sum', f'{prefix}경상자수': 'sum', 
                                                                    f'{variable_name}_ECLO':'sum'
                                                                    }).reset_index()

# (6)-5-2 Count the number of occurrences for each group and reset index
count = globals()[f'{variable_name}'].groupby(['구', '동가']).size().reset_index(name=f'{prefix}지역수')

# (6)-5-3 Merge the two DataFrames on '구' and '동가'
globals()[f'{variable_name}'] = pd.merge(grouped, count, on=['구', '동가'])

# 
# globals()[f'{variable_name}'] = globals()[f'{variable_name}'].groupby(['구','동가']).mean().reset_index()
# globals()[f'{variable_name}'] = globals()[f'{variable_name}'].groupby(['구','동가','법규위반']).mean().reset_index()

In [1146]:
globals()[f'{variable_name}']

,구,동가,화물차사고건수,화물차사망자수,화물차중상자수,화물차경상자수,truck_ECLO,화물차지역수
0,남구,대명동,5,1,4,4,42,1
1,남구,봉덕동,4,0,6,13,69,1
2,달서구,감삼동,4,0,4,2,26,1
3,달서구,두류동,8,0,8,1,43,2
4,달서구,신당동,4,0,6,2,36,1
5,달서구,용산동,8,1,7,4,57,2
6,달서구,월성동,5,0,5,4,37,1
7,달서구,이곡동,4,0,6,0,30,1
8,달서구,죽전동,4,1,3,3,34,1
9,달성군,다사읍,4,0,4,0,20,1


#### violt.csv

In [1162]:
# (6)-5-1 Average and initialize the values ​​of variables for rows where ‘동가’ and ‘법정동코드’ are the same
grouped = globals()[f'{variable_name}'].groupby(['구', '동가']).agg({
                                                                    f'{prefix}사고건수': 'sum',
                                                                    # jaywalk.csv
                                                                    # f'{prefix}발생건수': 'sum', 
                                                                    f'{prefix}사망자수': 'sum', 
                                                                    f'{prefix}중상자수': 'sum', f'{prefix}경상자수': 'sum', 
                                                                    f'{variable_name}_ECLO':'sum',
                                                                    '중앙선침범':'sum','신호위반':'sum'
                                                                    }).reset_index()

# (6)-5-2 Count the number of occurrences for each group and reset index
count = globals()[f'{variable_name}'].groupby(['구', '동가']).size().reset_index(name=f'{prefix}지역수')

# (6)-5-3 Merge the two DataFrames on '구' and '동가'
globals()[f'{variable_name}'] = pd.merge(grouped, count, on=['구', '동가'])

# 
# globals()[f'{variable_name}'] = globals()[f'{variable_name}'].groupby(['구','동가']).mean().reset_index()
# globals()[f'{variable_name}'] = globals()[f'{variable_name}'].groupby(['구','동가','법규위반']).mean().reset_index()

In [1163]:
globals()[f'{variable_name}']

,구,동가,법규위반사고건수,법규위반사망자수,법규위반중상자수,법규위반경상자수,violt_ECLO,중앙선침범,신호위반,법규위반지역수
0,남구,대명동,36,1,11,43,194,3,4,7
1,남구,봉덕동,12,0,11,21,118,3,0,3
2,달서구,감삼동,21,0,9,18,99,1,2,3
3,달서구,대곡동,16,0,4,23,89,0,2,2
4,달서구,대천동,3,0,2,6,28,1,0,1
...,...,...,...,...,...,...,...,...,...,...
62,중구,삼덕동1가,3,0,0,6,18,1,0,1
63,중구,삼덕동2가,5,0,3,4,27,1,0,1
64,중구,삼덕동3가,6,0,5,5,40,0,1,1
65,중구,태평로3가,7,0,4,3,29,0,1,1


# Save the data

In [1164]:
# (7) data save
INPUT_PATH = './data/external_open/bigdatamart/7/new/'
rename = '_new'
globals()[f'{variable_name}'].to_csv(f'{INPUT_PATH}{variable_name}{rename}.csv', index=False)

# Merge Data

In [1165]:
# (8) merge data
INPUT_PATH = './data/external_open/bigdatamart/new/'
rename = '_new'

# 사고 다발 지역 정보 / 빅데이터마트 7.안전
oldman = pd.read_csv(f'{INPUT_PATH}oldman{rename}.csv') # 보행노인사고
bicycle = pd.read_csv(f'{INPUT_PATH}bicycle{rename}.csv') # 자전거 
child = pd.read_csv(f'{INPUT_PATH}child{rename}.csv') # 보행어린이사고
drunk = pd.read_csv(f'{INPUT_PATH}drunk{rename}.csv') # 음주사고 
freezing = pd.read_csv(f'{INPUT_PATH}freezing{rename}.csv') # 결빙사고
jaywalking = pd.read_csv(f'{INPUT_PATH}jaywalking{rename}.csv') # 보행자무단횡단
motorcycle = pd.read_csv(f'{INPUT_PATH}motorcycle{rename}.csv') # 이륜차사고
pedstrians = pd.read_csv(f'{INPUT_PATH}pedstrians{rename}.csv') # 보행자사고 
schoolzone = pd.read_csv(f'{INPUT_PATH}schoolzone{rename}.csv') # 스쿨존내어린이사고
tmzon = pd.read_csv(f'{INPUT_PATH}tmzon{rename}.csv') # 연휴기간 
violt = pd.read_csv(f'{INPUT_PATH}violt{rename}.csv') # 법규위반  
truck = pd.read_csv(f'{INPUT_PATH}truck{rename}.csv') # 화물차 

In [1166]:
# (9) Merge two data frames based on ‘동가’ and ‘구’
merged_df = oldman.merge(bicycle, on=['동가', '구'], how='outer')
merged_df = merged_df.merge(child, on=['동가', '구'], how='outer')
merged_df = merged_df.merge(drunk, on=['동가', '구'], how='outer')
merged_df = merged_df.merge(freezing, on=['동가', '구'], how='outer')
merged_df = merged_df.merge(jaywalking, on=['동가', '구'], how='outer')
merged_df = merged_df.merge(motorcycle, on=['동가', '구'], how='outer')
merged_df = merged_df.merge(pedstrians, on=['동가', '구'], how='outer')
merged_df = merged_df.merge(schoolzone, on=['동가', '구'], how='outer')
merged_df = merged_df.merge(tmzon, on=['동가', '구'], how='outer')
merged_df = merged_df.merge(violt, on=['동가', '구'], how='outer')
merged_df = merged_df.merge(truck, on=['동가', '구'], how='outer')
result_df = merged_df.fillna(0)
result_df

,구,동가,보행노인사고사고건수,보행노인사고사망자수,보행노인사고중상자수,보행노인사고경상자수,oldman_ECLO,보행노인사고지역수,자전거사고사고건수,자전거사고사망자수,...,violt_ECLO,중앙선침범,신호위반,법규위반지역수,화물차사고건수,화물차사망자수,화물차중상자수,화물차경상자수,truck_ECLO,화물차지역수
0,남구,대명동,94.0,5.0,60.0,40.0,470.0,23.0,98.0,2.0,...,194.0,3.0,4.0,7.0,5.0,1.0,4.0,4.0,42.0,1.0
1,남구,봉덕동,40.0,1.0,27.0,13.0,184.0,10.0,33.0,2.0,...,118.0,3.0,0.0,3.0,4.0,0.0,6.0,13.0,69.0,1.0
2,남구,이천동,11.0,2.0,6.0,3.0,59.0,3.0,16.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,달서구,감삼동,6.0,0.0,5.0,1.0,28.0,2.0,85.0,0.0,...,99.0,1.0,2.0,3.0,4.0,0.0,4.0,2.0,26.0,1.0
4,달서구,두류동,32.0,1.0,21.0,11.0,148.0,9.0,77.0,1.0,...,130.0,1.0,3.0,4.0,8.0,0.0,8.0,1.0,43.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,동구,중대동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,55.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
120,서구,이현동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
121,중구,사일동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
122,동구,부동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,4.0,1.0,33.0,1.0


In [1167]:
# (10) save merge data
result_df.to_csv(f'./data/external_open/bigdatamart/7/new/bigdatamart_merge.csv', index=False)